<h2> More analysis</h2>
<p> In the previous notebook, we did some data munging, model selection, hyperparameter optimization and community analysis. In this notebook, we'll do a bit more analysis and also apply LDA to range estimation and species distribution modeling.</p>

In [1]:
from collections import Counter
from csv import reader
import descartes
import fiona
import gensim
from geopandas import GeoDataFrame
import geopy
from IPython.display import Image,display
import logging
import matplotlib.pyplot as plt
from mpl_toolkits import basemap
import mpl_toolkits
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import pyper as pr
import seaborn as sns
import shapely
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
import sys

%matplotlib inline

Couldn't import dot_parser, loading of dot files will not be possible.


/home/chris/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [12]:
# First, we'll load the dataframes we were using before:

speciesCommunityDF = pd.read_csv('dataframes/speciesCommunityDF.csv',index_col='Unnamed: 0')
communityLocationDF = pd.read_csv('dataframes/communityLocationDF.csv',index_col = 'verbatimCoordinates')



<h2> Another round of clustering</h2>
<p> We've got spatial maps of community prominence over the landscape, but what we'd really like to know is which physical locations are similar to each other with regard to the distribution over all communities! One way to do this is by clustering the physical locations by their distributions over communities. Then, the cluster labels should group together locations which have similar makeup of communities. We'll use the clustering algorithm [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) after a dimension reduction step using [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) as implemented in sklearn. Note that running TSNE on this many observations is expensive; TSNE has space complexity of $O(n^2)$ which makes large datasets infeasible. 12000 records is pushing the limit.

In [ ]:
theta = communityLocationDF.drop(['decimalLatitude','decimalLongitude'],1)
assert theta.shape[0] > theta.shape[1]  

# TSNE finds an embedding of the data which seeks to keep similar points near other similar points.
tsneModel = TSNE(n_components = 2, random_state = 827)
reducedTheta = tsneModel.fit_transform(theta)

# Let's get a look at the low-dimensional projection that TSNE came up with.
plt.scatter(reducedTheta)

dbscan = DBSCAN()
dbscan.fit(reducedTheta)
clusterLabels = dbscan.fit_predict(theta)
